In [1]:
%load_ext watermark


In [2]:
from downstream import dstream
from hstrat import hstrat
from hstrat import _auxiliary_lib as hstrat_aux
import pandas as pd


In [3]:
%watermark -diwmuv -iv


Last updated: 2025-08-31T00:23:37.080401+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.31.1

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.8.0-1031-azure
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

downstream: 1.14.3
hstrat    : 1.20.10
pandas    : 2.2.3

Watermark: 2.4.3



In [4]:
teeplot_subdir = "2025-05-09-reconstruction-quality"
teeplot_subdir


'2025-05-09-reconstruction-quality'

## Prep Data


In [5]:
true_phylo_df = pd.read_csv("https://osf.io/ypqvb/download")
raw_genome_df = pd.read_parquet("https://osf.io/4x7p8/download")


In [6]:
true_phylo_df = hstrat_aux.alifestd_prune_extinct_lineages_asexual(
    true_phylo_df,
)
true_phylo_df = hstrat_aux.alifestd_mark_leaves(true_phylo_df)
true_phylo_df = hstrat_aux.alifestd_collapse_unifurcations(
    true_phylo_df,
)
true_phylo_df


,id,ancestor_list,origin_time,destruction_time,num_orgs,tot_orgs,num_offspring,total_offspring,depth,taxon_label,ancestor_id,is_leaf
0,1,[none],0,0.0,0,1,1,160,0,cd613e30-d8f1-4adf-91b7-584a2265b1f5,1,False
63,64,[1],0,0.0,0,1,12,160,63,a49ffca6-af6c-4513-a3af-cb3e05983a75,1,False
194,13174,[64],0,0.0,0,1,2,149,194,0920845e-4336-4272-aea8-5e6719145c3a,64,False
227,16430,[13174],0,0.0,0,1,2,146,227,466f8f8d-7aa2-42e0-ac65-57d2162808ac,13174,False
245,18230,[16430],0,0.0,0,1,2,141,245,8b1a930d-d266-4617-b27b-82ba4cdcce77,16430,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3357,50249,[49511],0,inf,1,1,0,0,565,d74426e0-81be-4b45-9aaf-3934734221d4,49511,True
3358,50248,[50049],0,inf,1,1,0,0,566,9ccb9bad-3d53-4c83-a5e1-3126ce065307,50049,True
3359,50247,[49665],0,inf,1,1,0,0,565,538368a0-ff21-4bbc-87b8-863fff073778,49665,True
3360,50246,[49190],0,inf,1,1,0,0,565,e803075a-4b5d-4061-babe-073c075fc2cd,49190,True


In [7]:
raw_genome_df = raw_genome_df.loc[
    raw_genome_df["taxon_label"].isin(true_phylo_df["taxon_label"])
].copy()
raw_genome_df


,downstream_version,data_hex,taxon_label,dstream_algo,dstream_storage_bitoffset,dstream_storage_bitwidth,dstream_T_bitoffset,dstream_T_bitwidth,dstream_S,trait,is_fossil,downstream_exclude_exploded,downstream_validate_exploded,downstream_validate_unpacked
0,1.14.3,000000437cf5204a4c421d7636bb91c0,90b4d910-4f88-49de-a075-230acac98b47,dstream.steady_algo,32,64,0,32,64,-4.554103,True,None,None,None
1,1.14.3,000000437cf5204a4c421d76c8f196c0,ea5dbadc-10c7-4863-bd27-88870283af41,dstream.steady_algo,32,64,0,32,64,-4.717014,True,None,None,None
2,1.14.3,000000437cf5204a4c421d76b10586c0,8c41c7f7-332b-4695-aacf-b97975753f8d,dstream.steady_algo,32,64,0,32,64,-4.188195,True,None,None,None
3,1.14.3,000000437cf5204a4c429d76b0349dc0,ade776fb-1dd1-4aac-b098-eca9729664c9,dstream.steady_algo,32,64,0,32,64,-4.912682,True,None,None,None
4,1.14.3,000000437cf5204a4c421d7660236fc0,cf72ec23-ad6d-45f9-a6fc-af920c88fda3,dstream.steady_algo,32,64,0,32,64,-3.736534,True,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,1.14.3,0000032c7c16292149dbf7a76cf00040,1fefcc0b-4a07-4b20-8462-965761b51791,dstream.steady_algo,32,64,0,32,64,2.014674,False,None,None,None
156,1.14.3,0000032f7c16292148dbf7a714fb95c1,0ce5b4d9-91a6-4b98-9064-482d52a132b1,dstream.steady_algo,32,64,0,32,64,-18.747597,False,None,None,None
157,1.14.3,000003307c162921499bf7a70a58d6bf,cc9d4975-12f2-4999-ad04-d9edcd0278fe,dstream.steady_algo,32,64,0,32,64,-1.674562,False,None,None,None
158,1.14.3,0000032b5c37282149db6127cf8aebc0,93d620d2-4b67-4e05-aa24-7305285d9397,dstream.steady_algo,32,64,0,32,64,-7.360695,False,None,None,None


In [8]:
kwargs = dict(
    dstream_algo=eval(
        raw_genome_df["dstream_algo"].unique().item(),
        {"dstream": dstream},
    ),
    dstream_S=raw_genome_df["dstream_S"].unique().item(),
    dstream_storage_bitoffset=raw_genome_df["dstream_storage_bitoffset"]
    .unique()
    .item(),
    dstream_storage_bitwidth=raw_genome_df["dstream_storage_bitwidth"]
    .unique()
    .item(),
    dstream_T_bitoffset=raw_genome_df["dstream_T_bitoffset"].unique().item(),
    dstream_T_bitwidth=raw_genome_df["dstream_T_bitwidth"].unique().item(),
)

population = [
    hstrat.surf_from_hex(
        genome_hex,
        **kwargs,
    )
    for genome_hex in raw_genome_df["data_hex"]
]
len(population)


160

## Naive Reconstruction vs. Ground Truth


In [9]:
naive_df = hstrat.build_tree_trie(
    population,
    taxon_labels=raw_genome_df["taxon_label"],
    force_common_ancestry=True,
)
naive_df


/usr/local/lib/python3.10/dist-packages/alifedata_phyloinformatics_convert/_impl/phytrack_Systematcs.py:7: UserWarning: ImportWarning: phylotrackpy.Systematics import failed; inserting a no-op mock for Systematics. This is likely because phylotrackpy is not installed.
  warnings.warn(


,id,ancestor_list,origin_time,taxon_label,ancestor_id
0,0,[none],0.0,Root,0
32,32,[0],0.5,Inner+r=-1+d=A+uid=H2_kkpgSEV4YeAuw5I1vA,0
127,127,[32],132.5,Inner+r=131+d=B+uid=BWvvo5XjxTbEFaxADXVHCA,32
167,167,[127],172.5,Inner+r=171+d=A+uid=BRQjKGpuzDHzUmO0UZohBc,127
175,175,[127],180.5,Inner+r=179+d=B+uid=CIx4D2kH-WaUupVfPkCWFQ,127
...,...,...,...,...,...
518,518,[485],751.0,85827754-21df-4b02-9716-0c131018533b,485
519,519,[485],751.0,662dece9-0602-47c7-a2eb-6b7925b44b3e,485
520,520,[485],751.0,9eb2d484-4258-4e20-a88b-c85e5b6c793e,485
521,521,[485],751.0,d5cec49c-7e0e-4177-8d0d-85c3a7a150bc,485


In [10]:
hstrat_aux.alifestd_estimate_triplet_distance_asexual(
    naive_df,
    true_phylo_df,
    taxon_label_key="taxon_label",
)


0.03012820512820513

## Shortcut Reconstruction vs. Ground Truth


In [11]:
shortcut_df = hstrat.build_tree_searchtable(
    population,
    taxon_labels=raw_genome_df["taxon_label"],
    force_common_ancestry=True,
)
shortcut_df


,differentia,ancestor_id,id,dstream_data_id,rank,origin_time,taxon_label,ancestor_list
0,0,0,0,9223372036854775807,0,0,_inner_node,[none]
1,0,0,1,9223372036854775807,-64,-64,_inner_node,[0]
2,1,1,2,9223372036854775807,-63,-63,_inner_node,[1]
3,0,2,3,9223372036854775807,-62,-62,_inner_node,[2]
4,1,3,4,9223372036854775807,-61,-61,_inner_node,[3]
...,...,...,...,...,...,...,...,...
502,18446744073709551615,485,502,155,759,759,9611efea-6e50-4dcc-ab16-ea97e03f26e9,[485]
503,18446744073709551615,487,503,156,759,759,9eb2d484-4258-4e20-a88b-c85e5b6c793e,[487]
504,18446744073709551615,487,504,157,759,759,d5cec49c-7e0e-4177-8d0d-85c3a7a150bc,[487]
505,18446744073709551615,487,505,158,759,759,3d84465a-6167-4736-b983-64ae37568334,[487]


In [12]:
hstrat_aux.alifestd_estimate_triplet_distance_asexual(
    shortcut_df,
    true_phylo_df,
    taxon_label_key="taxon_label",
)


0.019058963668850508

## Naive Reconstruction vs. Shortcut Reconstruction


In [13]:
hstrat_aux.alifestd_estimate_triplet_distance_asexual(
    shortcut_df,
    naive_df,
    taxon_label_key="taxon_label",
)


0.005025125628140704